# Plot Group Traces
**Description:** This notebook visualizes the signals for all embryos within a group (across experiments). It specifically uses the **preprocessed** trace information which defaults to finding the first burst and trimming the trace and time 30 mins before that burst. These parameters can be changed in the Trace class. 

* All raw signals (active and structural)
* All dff traces
* All spectrograms

## Import Libraries

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import matplotlib as mpl
import numpy as np
from scipy.signal import savgol_filter

from golf_analysis import Group, Experiment, FrequencyAnalysis, myplots

## Load Data

Specify the location of the data by editing `folder` and entering experiments. All of the embryos in those experiments will be processed togehter.

In [ ]:
folder = "25C"
group_config = ["20250404_25C", "20240919_25C", "20240611_25C"]  # WT

experiments = {}

for exp in group_config:
    exp_path = Path.cwd().parent.joinpath("data", folder, exp)
    experiment_object = Experiment(exp_path)
    experiments[exp] = experiment_object

group = Group(folder, experiments)

## Configure Plot Style

Customize the Matplotlib plots with rcParams. For a full list of parameters see [matplotlib.rcParams](https://matplotlib.org/stable/api/matplotlib_configuration_api.html#default-values-and-styling)

In [ ]:
myplots.set_darkbg(False)

subplots_rc = {
    "figure.frameon": False,
    "figure.figsize": [20, 2.5],  # approzimate size of each individual subplot
    "lines.linewidth": 2,
    "font.family": "Arial",
    "font.size": 15,
    "xtick.major.size": 10,  # length
    "xtick.major.width": 2,
    "ytick.major.size": 10,  # length
    "ytick.major.width": 2,
    "axes.spines.top": False,
    "axes.spines.right": False,
    "axes.labelpad": 10,
    "axes.linewidth": 2.5,
}

spec_rc = {
    "figure.figsize": (35, 8),
    "figure.frameon": False,
    "font.family": "Arial",
    "font.size": 40,
    "axes.spines.top": False,
    "axes.spines.right": False,
    "xtick.major.size": 10,  # length
    "xtick.major.width": 2,
    "ytick.major.size": 10,  # length
    "ytick.major.width": 2,
    "axes.labelpad": 10,
    "axes.linewidth": 2.5,
}

mymap = mpl.colormaps["turbo"]
mymap.set_over("black")

## Plot Raw Signals

Plots the raw activity signal (right) and structural signal (left).

**plot_raw_signals**

* title: figure title

In [ ]:
for exp in group.experiments.values():
    myplots.plot_raw_signals(exp.embryos, subplots_rc, title=exp.name)

## Plot All DFF Traces

Plots the processed dff traces for all embryos.

**plot_traces**

optional parameters:
* color: line color
* xmin: x-axis minimum in minutes
* xmax: x-axis maximum in minutes
* ymin: y-axis minimum in minutes
* ymax: y-axis maximum in minutes

In [ ]:
for exp in group.experiments.values():
    myplots.plot_traces(
        exp.embryos, subplots_rc, title=exp.name, color="#0273b2ff", ymax=1.5
    )

## Plot All Spectrograms

Gets preprocessed dffs, then plots the magnitude of its STFT over time. The x-axis displays time (columns), y-axis displays frequency (rows), and color displays intensity. 

**preprocess_multi_dffs** 

optional parameters:
* duration: target number of frames to include
* onset_pad: number of frames to include before onset


**calculate_multi_STFTs**

optional parameters:
* fs: sampling rate
* fft_size: num frames in each segment
* noverlap: num frames to overlap between segements

**plot_specs** 

optional parameters:
* display_colorbar: True to display, False to hide
* xmin: x-axis minimum in minutes
* xmax: x-axis maximum in minutes
* ymin: y-axis minimum in minutes
* ymax: y-axis maximum in minutes

In [ ]:
for exp in group.experiments.values():
    myplots.plot_specs(
        exp.embryos, mymap, subplots_rc, title=exp.name, display_colorbar=False
    )

## Plot Average Spectrogram

Gets preprocessed dffs, then plots the average magnitude of their STFTs over time. The x-axis displays time (columns), y-axis displays frequency (rows), and color displays intensity. 

**calculate_STFTs**

optional parameters:
* fs: sampling rate
* fft_size: num frames in each segment
* noverlap: num frames to overlap between segements

**plot_spec** 

optional parameters:
* display_colorbar: True to display, False to hide
* xmin: x-axis minimum in minutes
* xmax: x-axis maximum in minutes
* ymin: y-axis minimum in minutes
* ymax: y-axis maximum in minutes

In [ ]:
f_all = []
t_all = []
Zxx_all = []

for exp in group.experiments.values():
    for emb in exp.embryos:
        dff = emb.trace.aligned_dff
        smoothed_dff = savgol_filter(dff, window_length=25, polyorder=4)
        f, t, Zxx = FrequencyAnalysis.calculate_STFT(smoothed_dff)
        f_all.append(f)
        t_all.append(t)
        Zxx_all.append(Zxx)

stft_magnitudes = np.abs(Zxx_all)
avg_magnitude = np.mean(stft_magnitudes, axis=0)
myplots.plot_spec(
    f, t, avg_magnitude, mymap, spec_rc, display_colorbar=False, ymax=0.015
)